### Word Group In Youtube

In [247]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [248]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [249]:
# language pair
lang_folder = "Arabic"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 28  # native word end index 28,200,1000,5000,10000, 20000, 40000

# word ratio
if word_end == 28 or word_end == 200:
    unique_word_ratio = 70
    include_wordgroup_ratio = 70
elif word_end == 1000 or word_end == 5000:
    unique_word_ratio = 80
    include_wordgroup_ratio = 80
elif word_end == 10000:
    unique_word_ratio = 85
    include_wordgroup_ratio = 85
else:
    unique_word_ratio = 90
    include_wordgroup_ratio = 90

# word usage
if word_end == 28 or word_end == 200 or word_end == 1000 or word_end == 5000 or word_end == 10000:
    word_usage_min = 10
    word_usage_max = 100
elif word_end == 20000:
    word_usage_min = 7
    word_usage_max = 100
else:
    word_usage_min = 5
    word_usage_max = 100

# use for manuel set
#unique_word_ratio = 70  # 28,200 = 70 ; 1000,5000 = 80 ; 10000 = 85 ; 20000,40000 = 90
#include_wordgroup_ratio = 70  # 28,200 = 70 ; 1000,5000 = 80 : 10000 = 85 ; 20000,40000 = 90
#word_usage_min = 10  # 28,200,1000,5000,10000 = 10 ; 20000 = 7 ; 40000 = 5
#word_usage_max = 100


# sentence ratio and time shift
adjust_sent_word_ratio = 15  #28,200 = 15 ; 1000,5000,10000 = 25 ; 20000 = 35 ; 40000 = 45
#adjust_text_word_ratio = 5  #28,200 = 5 ; 1000,5000,10000 = 25 ; 20000 = 35 ; 40000 = 45
shift = 0.3  # sentence time shift

# func select
if word_end == 28:
    sentence_word_count_number = 2
else:
    sentence_word_count_number = 5   
word_group_adjust_select = False  # True, False; True for selecting word group according to sentence_word_count_number in video_id text analysis
                                  # False for selecting word group according to max word group length 

# shared word frequency
word_frequency = True  # True, False

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"word_end:{word_end}\nfile_ext:{file_ext}\nunique_word_ratio:{unique_word_ratio}\n\
include_wordgroup_ratio:{include_wordgroup_ratio}\nword_usage_min:{word_usage_min}\nword_usage_max:{word_usage_max}")

word_end:28
file_ext:6
unique_word_ratio:70
include_wordgroup_ratio:70
word_usage_min:10
word_usage_max:100


In [250]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Language Level/Data/1-Word Group In Youtube Sentence"

Path(path).mkdir(parents=True, exist_ok=True)

In [251]:
def word_usage_result(word_list, df_target, target_column, target_opt_column, word_usage_min, word_usage_max):
    '''
    word_usage_result(word_list, df_ngram_pair, "threegram", "frequency", 1, 5) \n
    word_list is a list, df_target is a dateframe, target_column is df_target dataframe target column, \n
    target_opt_column is df_target dataframe opt_target column, \n
    word_usage_min and word_usage_max word usage condition.
    '''    
    word_num_dict = {}
    for i in word_list:
        word_num_dict[f"{i}"] = 0
    
    result_list_select = []
    var_list = []
    for i in range(len(df_target)):
        target_value = df_target.loc[i,f"{target_column}"]
        opt_value = df_target.loc[i,f"{target_opt_column}"]
        words = word_tokenize(target_value)   
        temp_list = [word for word in words]
        temp_list = temp_list + var_list
        # word count for max
        dict_list_count = Counter(temp_list)
        count_list = list(dict_list_count.values())
        # word count for min
        count_list2 = list(word_num_dict.values())
    
        if any([True if i>word_usage_max else False for i in count_list]) or not(any([True if j<word_usage_min else False for j in count_list2])):
            pass
        else:
            var_list = temp_list
            result_list_select.append([target_value,opt_value]) 
    
            for item2 in dict_list_count.items(): 
                word_num_dict[item2[0]] = item2[1]        
    df_result = pd.DataFrame(result_list_select, columns=[f"{target_column}",f"{target_opt_column}"])
    df_result.sort_values(by=f"{target_opt_column}", ascending=False, inplace=True)
    df_result.reset_index(drop=True, inplace=True)
    
    return df_result

In [252]:
def exract_word_group(text, word_list):
    '''
    exract_word_group(text, word_list): \n
    text is a string sentence, word_list occurs words 
    '''
    words = re.findall(r"\w+", text, re.UNICODE)
    index_list = []

    for i in range(len(words)):
        if words[i] in word_list:
            index_list.append(i)
        else:
            pass

    all_index_list = []
    var_index_list = []
    for j in range(len(index_list)):
        try:
            var1 = index_list[j] + 1  
            var2 = index_list[j+1]
        except:
            var1 = index_list[j] + 1  
            var2 = index_list[-1]
        if var1 == var2:
            var3 = index_list[j]
            var_index_list.append(var3)
        else:
            var3 = index_list[j]
            var_index_list.append(var3)
            var4 = list(var_index_list)
            all_index_list.append(var4)
            var_index_list = []

    text_list = []
    for k in max(all_index_list, key=len):  # any error convert k to i
        word = words[k]
        text_list.append(word)
        if len(text_list) >= sentence_word_count_number:
            text = " ".join(text_list)
        else:
            text = f"sentencte_word_count_less_than_{sentence_word_count_number}"
    
    return text 

In [253]:
def df_exract_word_group(df, source_text_column, opt_column, word_list, sent_len=False, sent_len_num=2):
    '''
    df_exract_word_group(df_adjust_text_ratio, sentence, video_id, word_list, sent_len=False, sent_len_num=2): \n
    df_adjust_text_ratio is a dataframe and it includes sentence and video_id columns. \n
    sentence is a string sentence. video_id is optional column value. word_list occurs words \n
    sent_len is sentence length condition. sent_len_num is sentence occurs how many word at least. 
    '''
    df.reset_index(drop=True, inplace=True)
    df_new = pd.DataFrame()
    for i in range(len(df)):
        source_text = df.loc[i,f"{source_text_column}"]
        opt_var = df.loc[i,f"{opt_column}"]
        words = re.findall(r"\w+", source_text, re.UNICODE)
        
        index_list = []
        for j in range(len(words)):
            if words[j] in word_list:
                index_list.append(j)
            else:
                pass

        all_index_list = []
        var_index_list = []
        for k in range(len(index_list)):
            try:
                var1 = index_list[k] + 1  
                var2 = index_list[k+1]
            except:
                var1 = index_list[k] + 1  
                var2 = index_list[-1]
            if var1 == var2:
                var3 = index_list[k]
                var_index_list.append(var3)
            else:
                var3 = index_list[k]
                var_index_list.append(var3)
                var4 = list(var_index_list)
                all_index_list.append(var4)
                var_index_list = []

        text_all_list = []
        for m in all_index_list:
            text_list = [] 
            for n in m:
                word = words[n]
                text_list.append(word)
                if sent_len:
                    if len(text_list) >= sent_len_num:
                        text = " ".join(text_list)
                        text_all_list.append(text)
                    else:
                        pass
                else:
                   text = " ".join(text_list)
                   text_all_list.append(text) 
            
        for search_string in text_all_list:
            #df_var["index"] = i
            #df_var["search_string"] = search_string
            #df_var["sentence"] = source_text
            #df_var["video_id"] = opt_var
            df_list = []
            df_list.append([i,search_string,source_text,opt_var])
            df_var = pd.DataFrame(df_list,columns=["index","search_string",f"{source_text_column}",f"{opt_column}"])
            df_new = pd.concat([df_new, df_var], axis=0)

    return df_new 

In [254]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    df.reset_index(drop=True, inplace=True)  # will test
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [255]:
def repetation_ratio(text):
    '''
    repetation_ratio(text) 
    text is a like as sentence and def compute repetation word ratio\n
    repetation_ratio("yok bir şey yok bir şey yok bir şey") 
    '''
    word_list = word_tokenize(text)
    word_num = len(word_list)
    word_unique_num = len(set(word_list))
    ratio = int((word_unique_num/word_num)*100)
    
    return ratio

In [256]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

In [ ]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [ ]:
# Option
if prefix_suffix:
    df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/{lang_folder.capitalize()}/{lang_folder.capitalize()}/{lang_folder.capitalize()}_{word_end}_Word_Prefix_Suffix_Custom_Result_Manuel.xlsx")
    df_word = df_word.loc[:,["word","frequency"]]
    df_word = pd.concat([df_word,df_word_select], axis=0)
    df_word.drop_duplicates(inplace=True)    
    df_word.sort_values(by="frequency", ascending=False, inplace=True)
    df_word.reset_index(drop=True, inplace=True)
else:
    df_word = df_word_select

if native_word:
    df_word
else:
    df_word = df_word.head(0)

df_word

,word,frequency
0,لا,5782309
1,من,5493373
2,أن,4348055
3,في,4324498
4,هذا,4155278
...,...,...
195,خلال,142614
196,الحقيقة,142231
197,بالتأكيد,142121
198,أنّ,138361


In [ ]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif","ass","fetisch","fetiche"]

In [ ]:
word_select = df_word["word"].values.tolist()
word_select_set = set(word_select)
disable_word_set = set(disable_list)

In [ ]:
word_list = list(word_select_set.difference(disable_word_set))
len(word_list)

200

#### Youtube Sentence Analysis

In [ ]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:00.000,00:00:04.000,مرحبا أصدقائي,Qx98dvR61cY
1,00:00:06.000,00:00:09.017,مضى اكثر من اسبوعين,Qx98dvR61cY
2,00:00:09.017,00:00:11.517,عندما هطلت الأمطار,Qx98dvR61cY
3,00:00:11.550,00:00:13.667,وحدثت فيضانات,Qx98dvR61cY
4,00:00:13.684,00:00:20.667,المعروف والشائع أن لون السيول اصفر طيني,Qx98dvR61cY
...,...,...,...,...
918116,00:00:34.396,00:00:36.396,قولتله خلاص سيبها,Hp54ZaJS1nI
918117,00:00:36.396,00:00:40.240,و حضنتها زى مانتم شايفين كده,Hp54ZaJS1nI
918118,00:00:45.588,00:00:48.627,حسيت بمشاعر جامدة جدا و هى فى حضنى,Hp54ZaJS1nI
918119,00:00:48.835,00:00:50.835,كأنك شايل طفل بالظبط,Hp54ZaJS1nI


In [ ]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [ ]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence.reset_index(inplace=True)  # adding index column
df_youtube_sentence

,index,start_time,end_time,sentence,video_id
0,0,0.000,4.000,مرحبا أصدقائي,Qx98dvR61cY
1,1,6.000,9.017,مضى اكثر من اسبوعين,Qx98dvR61cY
2,2,9.017,11.517,عندما هطلت الأمطار,Qx98dvR61cY
3,3,11.550,13.667,وحدثت فيضانات,Qx98dvR61cY
4,4,13.684,20.667,المعروف والشائع أن لون السيول اصفر طيني,Qx98dvR61cY
...,...,...,...,...,...
918116,918116,34.396,36.396,قولتله خلاص سيبها,Hp54ZaJS1nI
918117,918117,36.396,40.240,و حضنتها زى مانتم شايفين كده,Hp54ZaJS1nI
918118,918118,45.588,48.627,حسيت بمشاعر جامدة جدا و هى فى حضنى,Hp54ZaJS1nI
918119,918119,48.835,50.835,كأنك شايل طفل بالظبط,Hp54ZaJS1nI


In [ ]:
#df_youtube_sentence[df_youtube_sentence["index"] == 1287503]

In [ ]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list = manager.list()
word_set = set(word_list)
index_num = list(range(len(df_youtube_sentence)))

def sentence_word_ratio(index_num):
    index = df_youtube_sentence.loc[index_num,"index"]
    start = df_youtube_sentence.loc[index_num,"start_time"]
    end = df_youtube_sentence.loc[index_num,"end_time"]
    sentence = df_youtube_sentence.loc[index_num,"sentence"]
    videoid = df_youtube_sentence.loc[index_num,"video_id"]

    sent_word = re.findall(r"\w+", sentence, re.UNICODE)
    sent_word_set = set(sent_word)
    intersect_word = list(word_set.intersection(sent_word_set))
    different_word = list(sent_word_set.difference(word_set))
    word_ratio = round(((len(intersect_word)/(len(sent_word)+0.001))*100),1)
    different = ", ".join(different_word)
    intersect = ", ".join(intersect_word)

    result_list.append([index,start,end,sentence,videoid,word_ratio,different,intersect])  
    
   
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(sentence_word_ratio, index_num)

In [ ]:
df_sentence_ratio_result = pd.DataFrame(list(result_list), columns=["index","start_time","end_time","sentence","video_id","word_ratio","different_word","intersect_word"])
df_sentence_ratio_result.sort_values(by="index", ascending=True, inplace=True)
df_sentence_ratio_result.reset_index(drop=True, inplace=True)
df_sentence_ratio_result

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,0.000,4.000,مرحبا أصدقائي,Qx98dvR61cY,50.0,أصدقائي,مرحبا
1,1,6.000,9.017,مضى اكثر من اسبوعين,Qx98dvR61cY,25.0,"مضى, اسبوعين, اكثر",من
2,2,9.017,11.517,عندما هطلت الأمطار,Qx98dvR61cY,33.3,"هطلت, الأمطار",عندما
3,3,11.550,13.667,وحدثت فيضانات,Qx98dvR61cY,0.0,"وحدثت, فيضانات",
4,4,13.684,20.667,المعروف والشائع أن لون السيول اصفر طيني,Qx98dvR61cY,14.3,"السيول, المعروف, والشائع, اصفر, لون, طيني",أن
...,...,...,...,...,...,...,...,...
918116,918116,34.396,36.396,قولتله خلاص سيبها,Hp54ZaJS1nI,0.0,"سيبها, قولتله, خلاص",
918117,918117,36.396,40.240,و حضنتها زى مانتم شايفين كده,Hp54ZaJS1nI,16.7,"زى, حضنتها, شايفين, مانتم, كده",و
918118,918118,45.588,48.627,حسيت بمشاعر جامدة جدا و هى فى حضنى,Hp54ZaJS1nI,37.5,"بمشاعر, هى, حضنى, حسيت, جامدة","جدا, و, فى"
918119,918119,48.835,50.835,كأنك شايل طفل بالظبط,Hp54ZaJS1nI,0.0,"كأنك, شايل, بالظبط, طفل",


In [ ]:
df_sentence_ratio_result["different_word"] = df_sentence_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result["intersect_word"] = df_sentence_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result 

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,0.000,4.000,مرحبا أصدقائي,Qx98dvR61cY,50.0,أصدقائي,مرحبا
1,1,6.000,9.017,مضى اكثر من اسبوعين,Qx98dvR61cY,25.0,"مضى, اسبوعين, اكثر",من
2,2,9.017,11.517,عندما هطلت الأمطار,Qx98dvR61cY,33.3,"هطلت, الأمطار",عندما
3,3,11.550,13.667,وحدثت فيضانات,Qx98dvR61cY,0.0,"وحدثت, فيضانات",NaN
4,4,13.684,20.667,المعروف والشائع أن لون السيول اصفر طيني,Qx98dvR61cY,14.3,"السيول, المعروف, والشائع, اصفر, لون, طيني",أن
...,...,...,...,...,...,...,...,...
918116,918116,34.396,36.396,قولتله خلاص سيبها,Hp54ZaJS1nI,0.0,"سيبها, قولتله, خلاص",NaN
918117,918117,36.396,40.240,و حضنتها زى مانتم شايفين كده,Hp54ZaJS1nI,16.7,"زى, حضنتها, شايفين, مانتم, كده",و
918118,918118,45.588,48.627,حسيت بمشاعر جامدة جدا و هى فى حضنى,Hp54ZaJS1nI,37.5,"بمشاعر, هى, حضنى, حسيت, جامدة","جدا, و, فى"
918119,918119,48.835,50.835,كأنك شايل طفل بالظبط,Hp54ZaJS1nI,0.0,"كأنك, شايل, بالظبط, طفل",NaN


In [ ]:
df_adjust_sentence_ratio = df_sentence_ratio_result[df_sentence_ratio_result["word_ratio"] >= adjust_sent_word_ratio]
df_adjust_sentence_ratio.reset_index(inplace=True, drop=True)
df_adjust_sentence_ratio

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,0.000,4.000,مرحبا أصدقائي,Qx98dvR61cY,50.0,أصدقائي,مرحبا
1,1,6.000,9.017,مضى اكثر من اسبوعين,Qx98dvR61cY,25.0,"مضى, اسبوعين, اكثر",من
2,2,9.017,11.517,عندما هطلت الأمطار,Qx98dvR61cY,33.3,"هطلت, الأمطار",عندما
3,5,22.667,27.667,وهذه السيول ذهبت للنهر قبل أسبوعين,Qx98dvR61cY,16.7,"السيول, للنهر, أسبوعين, وهذه, ذهبت",قبل
4,6,27.667,34.434,ولكن الغريب بقت مياه تتحرك في الوادي وهذه المي...,Qx98dvR61cY,16.7,"المياه, تتحرك, بقت, كأنها, نقيه, مياه, فلتر, ا...","ولكن, في"
...,...,...,...,...,...,...,...,...
606631,918105,0.979,2.979,طلعت تجرى و جت عليا,Hp54ZaJS1nI,20.0,"طلعت, تجرى, جت, عليا",و
606632,918113,27.618,29.618,و انا حبيت حبوبة اوى,Hp54ZaJS1nI,40.0,"اوى, حبيت, حبوبة","انا, و"
606633,918117,36.396,40.240,و حضنتها زى مانتم شايفين كده,Hp54ZaJS1nI,16.7,"زى, حضنتها, شايفين, مانتم, كده",و
606634,918118,45.588,48.627,حسيت بمشاعر جامدة جدا و هى فى حضنى,Hp54ZaJS1nI,37.5,"بمشاعر, هى, حضنى, حسيت, جامدة","جدا, و, فى"


In [ ]:
df_adjust_sentence_ratio.loc[:,"search_string"] = df_adjust_sentence_ratio.loc[:,"sentence"].map(lambda x: exract_word_group(x, word_list))
df_adjust_sentence_ratio = df_adjust_sentence_ratio[~(df_adjust_sentence_ratio["search_string"]==f"sentencte_word_count_less_than_{sentence_word_count_number}")]
#df_adjust_sentence_ratio.reset_index(drop=True, inplace=True)
df_adjust_sentence_ratio

<ipython-input-229-ca15520b3270>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_adjust_sentence_ratio.loc[:,"search_string"] = df_adjust_sentence_ratio.loc[:,"sentence"].map(lambda x: exract_word_group(x, word_list))


,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word,search_string
30,46,22.643,23.586,ياسين أنا أريد أن أعرف عن نفسي,8mAA3d5GLSU,71.4,"ياسين, نفسي","أن, أعرف, أنا, أريد, عن",أنا أريد أن أعرف عن
61,83,200.142,203.744,و لكن قبل كل شيء سنتذوق من عندهم صينية فطوم,8mAA3d5GLSU,60.0,"سنتذوق, صينية, عندهم, فطوم","كل, لكن, من, شيء, و, قبل",و لكن قبل كل شيء
130,179,639.048,641.496,أكبر دليل على هذا أنه أنا بعد كل الأكل الذي أك...,8mAA3d5GLSU,58.3,"أكلته, أكبر, دليل, الأكل","بعد, كل, أنه, الذي, على, أنا, هذا",على هذا أنه أنا بعد كل
505,678,2783.230,2788.056,لا أعرف لكن لا أعتقد أن العثمانيين,ENsy8s3XBMU,71.4,العثمانيين,"أن, أعتقد, لكن, أعرف, لا",لا أعرف لكن لا أعتقد أن
510,683,2809.338,2816.384,مثلما نعلم جميعا أننا سنموت يوما ما لكننا نتصر...,ENsy8s3XBMU,46.7,"يوما, مثلما, جميعا, نعلم, لكننا, نتصرف, سنموت,...","ليس, أن, ما, كذلك, كما, لو, الأمر",كما لو أن الأمر ليس كذلك
...,...,...,...,...,...,...,...,...,...
602730,912018,352.468,356.168,يوم ولا لا لو انت رفعت قصير اخر مية وتمانين يو...,Q5mHykL9VSc,50.0,"وتمانين, قصير, اخر, رفعت, مية, يبدأ","لا, ان, لو, ولا, يوم, هو, انت",يوم ولا لا لو انت
603982,914026,415.321,418.246,من الرفق أن ندل الناس على شيء من هذا,ibEeZNx7UNk,66.7,"الرفق, ندل","أن, من, على, الناس, شيء, هذا",الناس على شيء من هذا
605016,915584,231.600,233.360,على الارجح أنك تعرف من أنا بالطبع تعرف,wo4AOBjefmQ,75.0,الارجح,"من, على, تعرف, أنك, أنا, بالطبع",أنك تعرف من أنا بالطبع تعرف
605208,915877,543.800,547.680,أن تقتل الناس في و و و,YhKVbFOXk18,57.1,تقتل,"في, أن, الناس, و",الناس في و و و


In [ ]:
df_word_group_time_loc_in_sent_result = word_group_time_loc(df_adjust_sentence_ratio, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,أنا أريد أن أعرف عن,22.800167,23.460267,ياسين أنا أريد أن أعرف عن نفسي,8mAA3d5GLSU
1,و لكن قبل كل شيء,200.142000,201.566047,و لكن قبل كل شيء سنتذوق من عندهم صينية فطوم,8mAA3d5GLSU
2,على هذا أنه أنا بعد كل,639.463698,640.572226,أكبر دليل على هذا أنه أنا بعد كل الأكل الذي أك...,8mAA3d5GLSU
3,لا أعرف لكن لا أعتقد أن,2783.230000,2786.636588,لا أعرف لكن لا أعتقد أن العثمانيين,ENsy8s3XBMU
4,كما لو أن الأمر ليس كذلك,2813.937472,2816.384000,مثلما نعلم جميعا أننا سنموت يوما ما لكننا نتصر...,ENsy8s3XBMU
...,...,...,...,...,...
4439,يوم ولا لا لو انت,352.468000,353.616276,يوم ولا لا لو انت رفعت قصير اخر مية وتمانين يو...,Q5mHykL9VSc
4440,الناس على شيء من هذا,416.539750,418.246000,من الرفق أن ندل الناس على شيء من هذا,ibEeZNx7UNk
4441,أنك تعرف من أنا بالطبع تعرف,232.063158,233.360000,على الارجح أنك تعرف من أنا بالطبع تعرف,wo4AOBjefmQ
4442,الناس في و و و,545.034545,547.680000,أن تقتل الناس في و و و,YhKVbFOXk18


In [ ]:
df_word_group_time_loc_in_sent_result["start_time"] = df_word_group_time_loc_in_sent_result["start_time"].apply(lambda x: (x-shift))
df_word_group_time_loc_in_sent_result["end_time"] = df_word_group_time_loc_in_sent_result["end_time"].apply(lambda x: (x+shift))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,أنا أريد أن أعرف عن,22.500167,23.760267,ياسين أنا أريد أن أعرف عن نفسي,8mAA3d5GLSU
1,و لكن قبل كل شيء,199.842000,201.866047,و لكن قبل كل شيء سنتذوق من عندهم صينية فطوم,8mAA3d5GLSU
2,على هذا أنه أنا بعد كل,639.163698,640.872226,أكبر دليل على هذا أنه أنا بعد كل الأكل الذي أك...,8mAA3d5GLSU
3,لا أعرف لكن لا أعتقد أن,2782.930000,2786.936588,لا أعرف لكن لا أعتقد أن العثمانيين,ENsy8s3XBMU
4,كما لو أن الأمر ليس كذلك,2813.637472,2816.684000,مثلما نعلم جميعا أننا سنموت يوما ما لكننا نتصر...,ENsy8s3XBMU
...,...,...,...,...,...
4439,يوم ولا لا لو انت,352.168000,353.916276,يوم ولا لا لو انت رفعت قصير اخر مية وتمانين يو...,Q5mHykL9VSc
4440,الناس على شيء من هذا,416.239750,418.546000,من الرفق أن ندل الناس على شيء من هذا,ibEeZNx7UNk
4441,أنك تعرف من أنا بالطبع تعرف,231.763158,233.660000,على الارجح أنك تعرف من أنا بالطبع تعرف,wo4AOBjefmQ
4442,الناس في و و و,544.734545,547.980000,أن تقتل الناس في و و و,YhKVbFOXk18


In [ ]:
df_word_group_time_loc_in_sent_result["start_time"] = df_word_group_time_loc_in_sent_result["start_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_sent_result["end_time"] = df_word_group_time_loc_in_sent_result["end_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,أنا أريد أن أعرف عن,23,24,ياسين أنا أريد أن أعرف عن نفسي,8mAA3d5GLSU
1,و لكن قبل كل شيء,200,202,و لكن قبل كل شيء سنتذوق من عندهم صينية فطوم,8mAA3d5GLSU
2,على هذا أنه أنا بعد كل,639,641,أكبر دليل على هذا أنه أنا بعد كل الأكل الذي أك...,8mAA3d5GLSU
3,لا أعرف لكن لا أعتقد أن,2783,2787,لا أعرف لكن لا أعتقد أن العثمانيين,ENsy8s3XBMU
4,كما لو أن الأمر ليس كذلك,2814,2817,مثلما نعلم جميعا أننا سنموت يوما ما لكننا نتصر...,ENsy8s3XBMU
...,...,...,...,...,...
4439,يوم ولا لا لو انت,352,354,يوم ولا لا لو انت رفعت قصير اخر مية وتمانين يو...,Q5mHykL9VSc
4440,الناس على شيء من هذا,416,419,من الرفق أن ندل الناس على شيء من هذا,ibEeZNx7UNk
4441,أنك تعرف من أنا بالطبع تعرف,232,234,على الارجح أنك تعرف من أنا بالطبع تعرف,wo4AOBjefmQ
4442,الناس في و و و,545,548,أن تقتل الناس في و و و,YhKVbFOXk18


In [ ]:
df_word_group_time_loc_in_sent_result.sort_values("search_string",key=lambda x:x.str.len(), inplace=True, ascending=False)
df_word_group_time_loc_in_sent_result.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,الآن قد يكون هناك بعض الحقيقة في ذلك حتى أن هن...,569,574,الآن قد يكون هناك بعض الحقيقة في ذلك حتى أن هن...,hVg78Xb6KlI
1,كنت تريد أن تكون من لا تريد أن تكون أنت فقط يم...,1361,1364,عن الذين كنت تريد أن تكون من لا تريد أن تكون أ...,Q4PgAY_qXKg
2,إذا كان الأمر كذلك يجب أن تعرف أن لا شيء كما يبدو,133,138,عندما تشعر بالغيرة فأنت مقتنع بأن شخصا ما لديه...,FmJZwVzRFTQ
3,نعم حسنا ليس هناك لا يوجد أحد هنا لا يمكنني,435,440,نعم حسنا ليس هناك لا يوجد أحد هنا لا يمكنني ال...,NCem4xje9Ss
4,ماذا ماذا ماذا أنت ما هي لا لا تفعل أنا آسف,882,887,ماذا ماذا ماذا أنت ما هي لا لا تفعل أنا آسف اا...,HnSK6tiiQZg
...,...,...,...,...,...
4439,و هو كيف و من,233,235,و هو كيف و من اين,8Itd8FrYclg
4440,مثل أو و و و,395,397,لشراء العملات الرقمية بشكل مستقل مثل أو و و و ...,frhNEeC4eK8
4441,في و و و و,314,318,هناك مراكز في و و و و,9hig9ulsjio
4442,ـ ـ ـ ـ ـ,31,36,ـ ـ ـ ـ ـ,PlpxSN2qDfg


In [ ]:
df_word_group_time_loc_in_sent_result["unique_word_ratio"] = df_word_group_time_loc_in_sent_result["search_string"].apply(lambda text: repetation_ratio(text))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id,unique_word_ratio
0,الآن قد يكون هناك بعض الحقيقة في ذلك حتى أن هن...,569,574,الآن قد يكون هناك بعض الحقيقة في ذلك حتى أن هن...,hVg78Xb6KlI,83
1,كنت تريد أن تكون من لا تريد أن تكون أنت فقط يم...,1361,1364,عن الذين كنت تريد أن تكون من لا تريد أن تكون أ...,Q4PgAY_qXKg,69
2,إذا كان الأمر كذلك يجب أن تعرف أن لا شيء كما يبدو,133,138,عندما تشعر بالغيرة فأنت مقتنع بأن شخصا ما لديه...,FmJZwVzRFTQ,91
3,نعم حسنا ليس هناك لا يوجد أحد هنا لا يمكنني,435,440,نعم حسنا ليس هناك لا يوجد أحد هنا لا يمكنني ال...,NCem4xje9Ss,90
4,ماذا ماذا ماذا أنت ما هي لا لا تفعل أنا آسف,882,887,ماذا ماذا ماذا أنت ما هي لا لا تفعل أنا آسف اا...,HnSK6tiiQZg,72
...,...,...,...,...,...,...
4439,و هو كيف و من,233,235,و هو كيف و من اين,8Itd8FrYclg,80
4440,مثل أو و و و,395,397,لشراء العملات الرقمية بشكل مستقل مثل أو و و و ...,frhNEeC4eK8,60
4441,في و و و و,314,318,هناك مراكز في و و و و,9hig9ulsjio,40
4442,ـ ـ ـ ـ ـ,31,36,ـ ـ ـ ـ ـ,PlpxSN2qDfg,20


In [ ]:
df_word_group_time_loc_in_sent_result_repeat_ratio = df_word_group_time_loc_in_sent_result[df_word_group_time_loc_in_sent_result["unique_word_ratio"] > unique_word_ratio]
df_word_group_time_loc_in_sent_result_repeat_ratio.drop_duplicates(subset=["search_string","start_time","end_time"], inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio.drop("unique_word_ratio", axis=1, inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio

<ipython-input-235-d6186b820154>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_in_sent_result_repeat_ratio.drop_duplicates(subset=["search_string","start_time","end_time"], inplace=True)
<ipython-input-235-d6186b820154>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_in_sent_result_repeat_ratio.drop("unique_word_ratio", axis=1, inplace=True)


,search_string,start_time,end_time,sentence,video_id
0,الآن قد يكون هناك بعض الحقيقة في ذلك حتى أن هن...,569,574,الآن قد يكون هناك بعض الحقيقة في ذلك حتى أن هن...,hVg78Xb6KlI
1,إذا كان الأمر كذلك يجب أن تعرف أن لا شيء كما يبدو,133,138,عندما تشعر بالغيرة فأنت مقتنع بأن شخصا ما لديه...,FmJZwVzRFTQ
2,نعم حسنا ليس هناك لا يوجد أحد هنا لا يمكنني,435,440,نعم حسنا ليس هناك لا يوجد أحد هنا لا يمكنني ال...,NCem4xje9Ss
3,ماذا ماذا ماذا أنت ما هي لا لا تفعل أنا آسف,882,887,ماذا ماذا ماذا أنت ما هي لا لا تفعل أنا آسف اا...,HnSK6tiiQZg
4,أوه اللعنة أوه اللعنة ما الذي يحدث يا إلهي,1036,1039,أوه اللعنة أوه اللعنة ما الذي يحدث يا إلهي,VSx7oL5PC7I
...,...,...,...,...,...
4127,جدا و هي ان لا,435,436,خطوة اخري مهمة جدا و هي ان لا تضيع وقتك في الف...,HhpJeKY0HpA
4128,في كل شيء و لم,600,602,فاذا خفتم في كل شيء و لم تستطيعوا ان تتحركوا,81zNtWfW0E8
4129,من ان هي في كل,22,23,من ان هي في كل منطقة في ليبيا بتتعمل بشوية بها...,OMx_uiTsgII
4130,أي أن كل من في,508,510,أي أن كل من في القائمة مواطنون عاديون,RCrVgQTbD6o


In [ ]:
df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] = df_word_group_time_loc_in_sent_result_repeat_ratio.apply(lambda x: int((len(str(x['search_string']))/len(str(x['sentence'])))*100) ,axis=1)
df_word_group_time_loc_in_sent_result_repeat_ratio

<ipython-input-236-e137816795b3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] = df_word_group_time_loc_in_sent_result_repeat_ratio.apply(lambda x: int((len(str(x['search_string']))/len(str(x['sentence'])))*100) ,axis=1)


,search_string,start_time,end_time,sentence,video_id,include_string_ratio
0,الآن قد يكون هناك بعض الحقيقة في ذلك حتى أن هن...,569,574,الآن قد يكون هناك بعض الحقيقة في ذلك حتى أن هن...,hVg78Xb6KlI,40
1,إذا كان الأمر كذلك يجب أن تعرف أن لا شيء كما يبدو,133,138,عندما تشعر بالغيرة فأنت مقتنع بأن شخصا ما لديه...,FmJZwVzRFTQ,41
2,نعم حسنا ليس هناك لا يوجد أحد هنا لا يمكنني,435,440,نعم حسنا ليس هناك لا يوجد أحد هنا لا يمكنني ال...,NCem4xje9Ss,76
3,ماذا ماذا ماذا أنت ما هي لا لا تفعل أنا آسف,882,887,ماذا ماذا ماذا أنت ما هي لا لا تفعل أنا آسف اا...,HnSK6tiiQZg,78
4,أوه اللعنة أوه اللعنة ما الذي يحدث يا إلهي,1036,1039,أوه اللعنة أوه اللعنة ما الذي يحدث يا إلهي,VSx7oL5PC7I,100
...,...,...,...,...,...,...
4127,جدا و هي ان لا,435,436,خطوة اخري مهمة جدا و هي ان لا تضيع وقتك في الف...,HhpJeKY0HpA,21
4128,في كل شيء و لم,600,602,فاذا خفتم في كل شيء و لم تستطيعوا ان تتحركوا,81zNtWfW0E8,31
4129,من ان هي في كل,22,23,من ان هي في كل منطقة في ليبيا بتتعمل بشوية بها...,OMx_uiTsgII,25
4130,أي أن كل من في,508,510,أي أن كل من في القائمة مواطنون عاديون,RCrVgQTbD6o,37


In [ ]:
df_word_group_time_loc_include_string_ratio = df_word_group_time_loc_in_sent_result_repeat_ratio[df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] > include_wordgroup_ratio]
df_word_group_time_loc_include_string_ratio.drop("include_string_ratio", axis=1, inplace=True)
df_word_group_time_loc_include_string_ratio.reset_index(drop=True, inplace=True)
df_word_group_time_loc_include_string_ratio

<ipython-input-237-ea709f90da2b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_include_string_ratio.drop("include_string_ratio", axis=1, inplace=True)


,search_string,start_time,end_time,sentence,video_id
0,نعم حسنا ليس هناك لا يوجد أحد هنا لا يمكنني,435,440,نعم حسنا ليس هناك لا يوجد أحد هنا لا يمكنني ال...,NCem4xje9Ss
1,ماذا ماذا ماذا أنت ما هي لا لا تفعل أنا آسف,882,887,ماذا ماذا ماذا أنت ما هي لا لا تفعل أنا آسف اا...,HnSK6tiiQZg
2,أوه اللعنة أوه اللعنة ما الذي يحدث يا إلهي,1036,1039,أوه اللعنة أوه اللعنة ما الذي يحدث يا إلهي,VSx7oL5PC7I
3,لقد قلت أنت لا يمكنك أن تفعل ذلك بي هناك,1242,1248,لقد قلت أنت لا يمكنك أن تفعل ذلك بي هناك أشياء,gqC0SbiQQc4
4,لذلك أنا لا أعرف أنني أعتقد إذا كنت تريد,295,299,لذلك أنا لا أعرف أنني أعتقد إذا كنت تريد,gkhlMGmoTWE
...,...,...,...,...,...
909,يا من له نعم لا,3974,3980,يا من له نعم لا تعد,7uPCY1lYZGQ
910,يا له من كم هذا,3384,3386,يا له من كم هذا فظ,Xlcy0eMQgvs
911,و لا حتى هذا أو,1437,1439,و لا حتى هذا أو ذاك,VfVomD2IVF8
912,ما هو لك هو لي,4125,4129,ما هو لك هو لي أيضا,Wiz60DqY9_c


In [ ]:
df_youtube_word_usage = word_usage_result(word_list, df_word_group_time_loc_include_string_ratio, "search_string", "video_id", word_usage_min, word_usage_max)
df_youtube_word_usage.drop_duplicates(inplace=True)
df_youtube_word_usage

,search_string,video_id
0,إلى الناس في هذا المكان,zsFBQBC0uLI
1,لكن في الحقيقة هو لم,zZtFDqdJuOU
2,حسنا لا يمكنني أكثر من,zUdW5hQzRDw
3,كل شيء بخير أعتقد ذلك,zUdW5hQzRDw
4,لا أعرف حتى إذا كان لدي واحد من,zUdW5hQzRDw
...,...,...
568,شيء آخر إذا لم يكن لديك أي شيء,1kvVhiOP0w8
569,ولا أريد أن يحدث أي شيء,1kvVhiOP0w8
570,أو حتى إنه لم يكن هناك,0aD7YI83YNM
571,كيف يكون هذا الأمر جيد,0PeXwP5myf8


In [ ]:
df_youtube_word_usage_merge = pd.merge(df_word_group_time_loc_include_string_ratio,df_youtube_word_usage,how="inner",on=["search_string","video_id"])
df_youtube_word_usage_merge.drop_duplicates(subset=["search_string","video_id"],inplace=True)
df_youtube_word_usage_merge.reset_index(drop=True, inplace=True)
df_youtube_word_usage_merge

,search_string,start_time,end_time,sentence,video_id
0,نعم حسنا ليس هناك لا يوجد أحد هنا لا يمكنني,435,440,نعم حسنا ليس هناك لا يوجد أحد هنا لا يمكنني ال...,NCem4xje9Ss
1,ماذا ماذا ماذا أنت ما هي لا لا تفعل أنا آسف,882,887,ماذا ماذا ماذا أنت ما هي لا لا تفعل أنا آسف اا...,HnSK6tiiQZg
2,أوه اللعنة أوه اللعنة ما الذي يحدث يا إلهي,1036,1039,أوه اللعنة أوه اللعنة ما الذي يحدث يا إلهي,VSx7oL5PC7I
3,لقد قلت أنت لا يمكنك أن تفعل ذلك بي هناك,1242,1248,لقد قلت أنت لا يمكنك أن تفعل ذلك بي هناك أشياء,gqC0SbiQQc4
4,لذلك أنا لا أعرف أنني أعتقد إذا كنت تريد,295,299,لذلك أنا لا أعرف أنني أعتقد إذا كنت تريد,gkhlMGmoTWE
...,...,...,...,...,...
558,مع أنه في كل مكان,15,17,مع أنه في كل مكان,fXukytNqkt4
559,يا إلهي و سيدي و,731,739,يا إلهي و سيدي و ربي,IHEwci25a5Q
560,ما إذا كان كل ما,1294,1296,ما إذا كان كل ما تريده,AFqZbxxIGK4
561,أنا هنا يا في في,4868,4870,أنا هنا يا في في,7ONEprXyxcs


In [ ]:
df_youtube_word_usage_merge["video_url"] = "https://www.youtube.com/watch?v="+df_youtube_word_usage_merge['video_id'].map(str)+"&t="+df_youtube_word_usage_merge['start_time'].map(str)+"s"
df_youtube_word_usage_merge

,search_string,start_time,end_time,sentence,video_id,video_url
0,نعم حسنا ليس هناك لا يوجد أحد هنا لا يمكنني,435,440,نعم حسنا ليس هناك لا يوجد أحد هنا لا يمكنني ال...,NCem4xje9Ss,https://www.youtube.com/watch?v=NCem4xje9Ss&t=...
1,ماذا ماذا ماذا أنت ما هي لا لا تفعل أنا آسف,882,887,ماذا ماذا ماذا أنت ما هي لا لا تفعل أنا آسف اا...,HnSK6tiiQZg,https://www.youtube.com/watch?v=HnSK6tiiQZg&t=...
2,أوه اللعنة أوه اللعنة ما الذي يحدث يا إلهي,1036,1039,أوه اللعنة أوه اللعنة ما الذي يحدث يا إلهي,VSx7oL5PC7I,https://www.youtube.com/watch?v=VSx7oL5PC7I&t=...
3,لقد قلت أنت لا يمكنك أن تفعل ذلك بي هناك,1242,1248,لقد قلت أنت لا يمكنك أن تفعل ذلك بي هناك أشياء,gqC0SbiQQc4,https://www.youtube.com/watch?v=gqC0SbiQQc4&t=...
4,لذلك أنا لا أعرف أنني أعتقد إذا كنت تريد,295,299,لذلك أنا لا أعرف أنني أعتقد إذا كنت تريد,gkhlMGmoTWE,https://www.youtube.com/watch?v=gkhlMGmoTWE&t=...
...,...,...,...,...,...,...
558,مع أنه في كل مكان,15,17,مع أنه في كل مكان,fXukytNqkt4,https://www.youtube.com/watch?v=fXukytNqkt4&t=15s
559,يا إلهي و سيدي و,731,739,يا إلهي و سيدي و ربي,IHEwci25a5Q,https://www.youtube.com/watch?v=IHEwci25a5Q&t=...
560,ما إذا كان كل ما,1294,1296,ما إذا كان كل ما تريده,AFqZbxxIGK4,https://www.youtube.com/watch?v=AFqZbxxIGK4&t=...
561,أنا هنا يا في في,4868,4870,أنا هنا يا في في,7ONEprXyxcs,https://www.youtube.com/watch?v=7ONEprXyxcs&t=...


In [ ]:
df_youtube_word_usage_merge = df_youtube_word_usage_merge.head(2000)
df_youtube_word_usage_merge.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.csv", index=False)

In [ ]:
df_youtube_word_usage_merge = df_youtube_word_usage_merge.head(700)
df_youtube_word_usage_merge.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.xlsx", index=False)

In [ ]:
#df_word_group_time_loc_in_sent_result = df_word_group_time_loc_in_sent_result.head(1000000)
#df_word_group_time_loc_in_sent_result.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.xlsx", index=False)

#### Copy Move And Delete

In [ ]:
output_file = glob.glob(f"{lang_folder.capitalize()}_Word_Group*{file_ext}.*")
output_file

['Arabic_Word_Group_Max_In_Youtube_Sentence_200_Word6.csv',
 'Arabic_Word_Group_Max_In_Youtube_Sentence_200_Word6.xlsx']

In [ ]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass